<a href="https://colab.research.google.com/github/grupomoura/ai_projects/blob/main/My_own_convnet_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
-rw-r--r-- 1 root root 66 May  8 22:44 kaggle.json


In [ ]:
!kaggle datasets download -d tommyngx/kneeoa

 99% 201M/204M [00:02<00:00, 99.6MB/s]
100% 204M/204M [00:02<00:00, 96.4MB/s]


In [ ]:
!cp /content/drive/MyDrive/Kaggle/Tumor-Cerebral_data-2class.zip .

In [ ]:
!unzip /content/drive/MyDrive/Kaggle/Tumor-Cerebral_data-2class.zip #/content/kneeoa.zip

Archive:  /content/drive/MyDrive/Kaggle/Tumor-Cerebral_data-2class.zip
   creating: Tumor Cerebral_data/
   creating: Tumor Cerebral_data/Testing/
   creating: Tumor Cerebral_data/Testing/no_tumor/
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(1).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(10).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(100).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(101).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(102).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(103).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(104).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(11).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(12).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(13).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(14).jpg  
  inflating: Tumor Cerebral_data/Testing/no_tumor/image(15).jpg  
  infl

# Imports

In [ ]:
# IMPORT MODULES
import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from imblearn.over_sampling import RandomOverSampler 

#from keras.applications import ResNet50

import os

# Variáveis e verificação 

In [ ]:
PATH = "/content/Tumor Cerebral_data/"
print(os.listdir(PATH))
target_size=(224, 224)
img_size=224
model_name='MyOwnConvnet'
n_class=2


['Training', 'Testing']


In [ ]:
train_dir = PATHtrain
validation_dir = PATHvalid
test_dir = PATHtest
batch_size = 10

#train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  ) 
                                  #fill_mode='nearest'
                                  #vertical_flip=False

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,target_size=target_size,batch_size=batch_size)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,target_size=target_size,batch_size=batch_size)
test_generator = test_datagen.flow_from_directory(
    test_dir,target_size=target_size,batch_size=batch_size)

Found 2870 images belonging to 2 classes.
Found 394 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [ ]:
#load the training data
PATHtrain = train_datagen.flow_from_directory(
    PATH,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True)
#load the training data
val_suffle=False
PATHvalid = train_datagen.flow_from_directory(
    PATH,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',shuffle=val_suffle)


"""# Check content of the dirs

PATHtrain = PATH + 'Training/'
print(len(os.listdir(PATHtrain)), " TRAIN Directories of photos")
Labels = os.listdir(PATHtrain)
sig = 0
for label in sorted(Labels):
    print(label,len(os.listdir(PATHtrain + label +'/')))
    sig = sig + len(os.listdir(PATHtrain + label +'/'))

print("Total TRAIN photos ", sig)
print("_"*50)

PATHvalid = PATH + 'Testing/'
print(len(os.listdir(PATHvalid)), " VALID Directories of photos")
Labels = os.listdir(PATHvalid)
sig = 0
for label in sorted(Labels):
    print(label,len(os.listdir(PATHvalid + label +'/')))
    sig = sig + len(os.listdir(PATHvalid + label +'/'))

print("Total Validation photos ", sig)
print("_"*50)

PATHtest = PATH + 'Testing/'
print(len(os.listdir(PATHtest)), " TEST Directories of photos")
Labels = os.listdir(PATHtest)
sig = 0
for label in sorted(Labels):
    print(label,len(os.listdir(PATHtest + label +'/')))
    sig = sig + len(os.listdir(PATHtest + label +'/'))

print("Total Testing photos ", sig)
print("_"*50)"""

Found 3264 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


'# Check content of the dirs\n\nPATHtrain = PATH + \'Training/\'\nprint(len(os.listdir(PATHtrain)), " TRAIN Directories of photos")\nLabels = os.listdir(PATHtrain)\nsig = 0\nfor label in sorted(Labels):\n    print(label,len(os.listdir(PATHtrain + label +\'/\')))\n    sig = sig + len(os.listdir(PATHtrain + label +\'/\'))\n\nprint("Total TRAIN photos ", sig)\nprint("_"*50)\n\nPATHvalid = PATH + \'Testing/\'\nprint(len(os.listdir(PATHvalid)), " VALID Directories of photos")\nLabels = os.listdir(PATHvalid)\nsig = 0\nfor label in sorted(Labels):\n    print(label,len(os.listdir(PATHvalid + label +\'/\')))\n    sig = sig + len(os.listdir(PATHvalid + label +\'/\'))\n\nprint("Total Validation photos ", sig)\nprint("_"*50)\n\nPATHtest = PATH + \'Testing/\'\nprint(len(os.listdir(PATHtest)), " TEST Directories of photos")\nLabels = os.listdir(PATHtest)\nsig = 0\nfor label in sorted(Labels):\n    print(label,len(os.listdir(PATHtest + label +\'/\')))\n    sig = sig + len(os.listdir(PATHtest + label 

In [ ]:
# Check the photos and their labels 
TestNum = 8
diag = 'tumor'

image_dir = PATHtrain +'/'+diag+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=target_size)
imgplot = plt.imshow(img)
print("TRAIN ",diag," photo number ", TestNum)
plt.show()

image_dir = PATHvalid +'/'+diag+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=target_size)
imgplot = plt.imshow(img)
print("VALID ",diag," photo number ", TestNum)
plt.show()

image_dir = PATHtest +'/'+diag+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=target_size)
imgplot = plt.imshow(img)
print("TEST ",diag," photo number ", TestNum)
plt.show()

TypeError: ignored

# Image generator

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

{'no_tumor': 0, 'tumor': 1}
{'no_tumor': 0, 'tumor': 1}
{'no_tumor': 0, 'tumor': 1}


In [ ]:
# Model save
my_callbacks = [
                #ModelCheckpoint(filepath='/content/drive/MyDrive/IA_TRAINABLE/callback_'+model_name+'_Osteo.h5', save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True),
                ReduceLROnPlateau(monitor="loss", mode="auto", factor=0.002, patience=2, min_lr=0.0001),
                EarlyStopping(monitor="accuracy", patience=10, restore_best_weights=True)
                ]


# My own convnet

In [ ]:
# MODEL
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal')) #kernel_initializer='he_normal'
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.00001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))
print(model.summary())

"""model = models.Sequential()
model.add(layers.Conv2D(512, (3, 3), activation='relu',
input_shape=(img_size, img_size, 3)))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(n_class, activation='sigmoid')) 
print(model.summary())"""

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 24, 24, 128)      

"model = models.Sequential()\nmodel.add(layers.Conv2D(512, (3, 3), activation='relu',\ninput_shape=(img_size, img_size, 3)))\nmodel.add(layers.Dropout(0.5))\nmodel.add(layers.BatchNormalization())\nmodel.add(layers.Conv2D(512, (3, 3), activation='relu'))\nmodel.add(layers.BatchNormalization())\nmodel.add(layers.Conv2D(256, (3, 3), activation='relu'))\nmodel.add(layers.BatchNormalization())\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Flatten())\nmodel.add(layers.Dropout(0.5))\nmodel.add(layers.Dense(64, activation='relu'))\nmodel.add(layers.Dropout(0.5))\nmodel.add(layers.Dense(n_class, activation='sigmoid')) \nprint(model.summary())"

In [ ]:
model.compile(optimizer=optimizers.Adam(lr=0.00001), #lr=1e-4
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model compiled!")

Model compiled!


In [ ]:
history = model.fit(train_generator,
                              epochs=100,
                              steps_per_epoch = train_generator.n // batch_size,
                              validation_data = validation_generator,
                              validation_steps = validation_generator.n // batch_size,
                              callbacks = my_callbacks
                    )

Epoch 1/100
287/287 [==============================] - 34s 117ms/step - loss: 0.5092 - accuracy: 0.8401 - val_loss: 0.6649 - val_accuracy: 0.7359
Epoch 2/100
287/287 [==============================] - 33s 116ms/step - loss: 0.4724 - accuracy: 0.8577 - val_loss: 0.6549 - val_accuracy: 0.7333
Epoch 3/100
287/287 [==============================] - 33s 116ms/step - loss: 0.4653 - accuracy: 0.8569 - val_loss: 0.6537 - val_accuracy: 0.7359
Epoch 4/100
287/287 [==============================] - 33s 116ms/step - loss: 0.4567 - accuracy: 0.8550 - val_loss: 0.6530 - val_accuracy: 0.7385
Epoch 5/100
287/287 [==============================] - 34s 120ms/step - loss: 0.4234 - accuracy: 0.8637 - val_loss: 0.6711 - val_accuracy: 0.7359
Epoch 6/100
287/287 [==============================] - 35s 122ms/step - loss: 0.3969 - accuracy: 0.8644 - val_loss: 0.6948 - val_accuracy: 0.7154
Epoch 7/100
287/287 [==============================] - 35s 122ms/step - loss: 0.3611 - accuracy: 0.8585 - val_loss: 0.9644 -

In [ ]:
# SAVE model

model.save('/content/drive/MyDrive/IA_TRAINED/OSTEOPOROSES/OsteoOwnConvnet.h5')
print("OsteoOwnConvnet.h5 was saved")

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=3561 // batch_size, verbose=1)
print('test acc:', test_acc)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()